# PLI v0 Demo (Modeling Notebook)

이 노트북은 **가중치 곱셈 기반 PLI(v0)** 데모입니다.  
- 환경적 요인: 날씨(기온)  
- 상황적 요인: 연패, 실책, 대타/대주자, 고의4구  
- 개인적 요인: 컨디션(최근5경기/시즌)  
- **상황 레벨**: 이닝/아웃/주자/점수차 기반 (0~11점 → L1~L4)

> 목표: **WPA × (모든 가중치의 곱)** → PLI 산출  
> WE/WPA, WAR, 파크팩터는 후속 버전에서 붙일 수 있도록 자리만 남겨둠.


In [1]:
# Imports
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, Any
import math
import pandas as pd
import numpy as np
from IPython.display import display
from typing import Optional

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)
wpa = pd.read_csv('../../data/kbo_we_matrix_filtered.csv')
stadium = pd.read_csv('../../data/stadium.csv')
stadium.head()

,stadium,REG_ID,type,city,parkfactor,weather,scode
0,창원NC파크,11H20301,C,창원,1.10,"""nx"": 89, ""ny"": 76",CW
1,수원KT위즈파크,11B20601,C,수원,1.00,"""nx"": 60, ""ny"": 121",SW
2,대구삼성라이온즈파크,11H10701,C,대구,1.50,"""nx"": 90, ""ny"": 90",DG
3,인천SSG랜더스필드,11B20201,C,인천,1.45,"""nx"": 55, ""ny"": 124",MH
4,고척스카이돔,NaN,NaN,서울,0.82,NaN,GC


## 날씨 불러오기

In [41]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

### 사용 예시

In [ ]:
# 비밀키는 안전한 곳에서 꺼내서 넘겨줘!
AUTH = "iVHg1Rm_SGuR4NUZv3hrLw"

# 코드로
res = get_stadium_temperature("JS", target_hour=18, auth_key=AUTH)

# 가중치까지
w = batting_weather_weight(res["temperature"])
print("weather weight =", w)

weather weight = 1.18


## 경기장 파크팩터

In [46]:
STADIUM_PARK = dict(zip(stadium["scode"], stadium["parkfactor"]))

def parkfactor_weight(parkfactor: float | None, gamma: float = 0.5,
                      cap_low: float = 0.85, cap_high: float = 1.25) -> float:
    """
    parkfactor ~ 1.00: 중립. 높을수록 타자친화.
    gamma: 민감도 계수 (0.5 → 루트 반영)
    """
    if parkfactor is None:
        return 1.0
    w = 1.0 + gamma * (float(parkfactor) - 1.0)
    return max(min(w, cap_high), cap_low)

## 시즌 기록 가져오기

In [2]:
import os, pathlib, sys
print("CWD:", os.getcwd())
print("sys.path head:", sys.path[:3])

CWD: /Users/yubin/Desktop/2025/dev/Back/ballrae_backend/pli_api
sys.path head: ['/app', '/opt/homebrew/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12']


In [3]:
import os, sys, pathlib
PROJ_ROOT = pathlib.Path("/Users/yubin/Desktop/2025/dev/Back")
sys.path.insert(0, str(PROJ_ROOT))

# 1) 먼저 .env 로드
try:
    from dotenv import load_dotenv
    load_dotenv(PROJ_ROOT / ".env")
except Exception:
    pass

# 2) 그 다음에 DB_HOST 덮어쓰기 (중요!)
os.environ["DB_HOST"] = "127.0.0.1"
os.environ["DJANGO_SETTINGS_MODULE"] = "ballrae_backend.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

# 확인
from django.conf import settings
print("DB HOST:", settings.DATABASES["default"]["HOST"])

DB HOST: 127.0.0.1


In [14]:
from ballrae_backend.players.models import Batter
batter = Batter.objects.filter(id="1377").first()

print(batter)

Batter object (1377)


In [13]:
from ballrae_backend.players.models import BatterRecent
BatterRecent.objects.all().values()

<QuerySet [{'id': 3, 'batter_id': 1377, 'ab': 0, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 45, 210244)}, {'id': 2, 'batter_id': 1376, 'ab': 0, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 44, 504535)}, {'id': 5, 'batter_id': 1379, 'ab': 1, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 46, 549991)}, {'id': 6, 'batter_id': 1380, 'ab': 3, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 47, 164076)}, {'id': 7, 'batter_id': 1381, 'ab': 0, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 47, 829847)}, {'id': 8, 'batter_id': 1382, 'ab': 6, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 52, 756523)}, {'id': 9, 'batter_id': 1383, 'ab': 10, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 53, 346451)}, {'id': 10, 'batter_id': 1384, 'ab': 6, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 53, 941969)}, {'id': 11, 'batter_id': 1385, 'ab': 12, 'hits': 0, 'updated

In [15]:
from ballrae_backend.players.models import BatterRecent
batterRecent = BatterRecent.objects.filter(batter=batter).first()

print(batterRecent)

BatterRecent object (3)


## 컨디션 지수

In [16]:
# 컨디션 지수 계산
def calculate_condition(pcode: str, season: int = 2025):
    from ballrae_backend.games.models import Player
    from ballrae_backend.players.models import Batter, Pitcher, BatterRecent
    
    try:
        player = Player.objects.get(pcode=pcode)
        
        if player.position == 'B':  # 타자
            try:
                batter = Batter.objects.get(player=player, season=season)
                recent = BatterRecent.objects.get(batter=batter)
                # 타율 계산
                ab = batter.ab or 0
                hits = (batter.singles or 0) + (batter.doubles or 0) + (batter.triples or 0) + (batter.homeruns or 0)
                
                condition = (recent.hits/recent.ab)/(hits/ab)
                result = 1.0 + (condition - 1.0) * 0.5
                return result

            except Batter.DoesNotExist:
                return None
                
        elif player.position == 'P':  # 투수
            return None
                
    except Player.DoesNotExist:
        return None

In [20]:
calculate_condition(78288)

0.8207070707070707

## 가중치 함수 (v0)

In [ ]:
def streak_weight(loss_streak: Optional[int]) -> float:
    if loss_streak is None or loss_streak <= 2: return 1.0
    if loss_streak <= 4: return 0.95
    if loss_streak <= 6: return 0.90
    return 0.85

def pinch_weight(flag: bool) -> float:
    return 1.8 if flag else 1.0

def ibb_focus_weight(flag: bool) -> float:
    return 2.0 if flag else 1.0

def error_momentum_weight(flag: bool, next_batter: bool=False) -> float:
    if not flag: return 1.0
    return 1.1 if next_batter else 1.3

def combine_weights(*ws: float) -> float:
    v = 1.0
    for w in ws:
        if w and w > 0:
            v *= float(w)
    return v


## 상황 레벨 (0~11점 → L1~L4)

In [5]:
def calculate_situation_level(inning: int, outs: int, runners_code: str, score_diff: int):
    total = 0
    # 이닝
    if inning >= 9: total += 3
    elif inning == 8: total += 2
    elif inning == 7: total += 1
    # 아웃
    if outs == 0: total += 2
    elif outs == 1: total += 1
    # 주자
    runner_scores = {
        '000':0,'100':1,'010':2,'110':2,'001':3,'101':3,'011':3,'111':3
    }
    total += runner_scores.get(runners_code, 0)
    # 점수차(절대값)
    d = abs(score_diff)
    if d == 0: total += 3
    elif d == 1: total += 2
    elif d <= 3: total += 1
    # 레벨
    if total >= 9: return 4, "최대 위기/방화", total
    if total >= 6: return 3, "중요한 승부처", total
    if total >= 3: return 2, "관리 필요", total
    return 1, "일상적 교체", total


## PLI 계산기

In [ ]:
def compute_pli_row(row: pd.Series) -> Dict[str, Any]:
    # 상황 레벨
    level, desc, score = calculate_situation_level(
        int(row['inning']), int(row['outs']), str(row['runners_code']), int(row['score_diff'])
    )
    # 가중치
    w_env = batting_weather_weight(row.get('temp_c'))
    w_situ = combine_weights(
        streak_weight(row.get('loss_streak')),
        pinch_weight(bool(row.get('pinch_event', False))),
        ibb_focus_weight(bool(row.get('intentional_walk', False))),
        error_momentum_weight(bool(row.get('error_happened', False)), bool(row.get('next_batter_after_error', False)))
    )
    pf = STADIUM_PARK.get('scode')
    w_park = parkfactor_weight(pf)
    w_total = combine_weights(w_env, w_situ, w_park)
    pli = float(row['base_wpa']) * w_total
    return {
        "pli": round(pli, 4),
        "w_env": round(w_env, 4),
        "w_situ": round(w_situ, 4),
        "w_total": round(w_total, 4),
        "level": level, "level_desc": desc, "level_score": score
    }


## WPA

In [ ]:
# adapters/pli_adapter.py
import re
from typing import Dict, Any, List, Optional

KW_PINCH = ("대타", "대주자")
KW_IBB = ("고의")
KW_ERROR = ("실책",)

def runners_code_from_onbase(on_base: Dict[str, str]) -> str:
    """
    on_base 예: {"base1":"0","base2":"9","base3":"0"}
    '0'이 아니면 1(점유)로 본다.
    반환: 'XYZ' (1루-2루-3루) → '101' 등
    """
    def occ(v: Optional[str]) -> str:
        return "1" if (v is not None and str(v) != "0") else "0"
    b1 = occ(on_base.get("base1"))
    b2 = occ(on_base.get("base2"))
    b3 = occ(on_base.get("base3"))
    return f"{b1}{b2}{b3}"

def parse_score_pair(score: str, fmt: str = "home:away") -> (int, int):
    """score '4:9' → (home, away)"""
    a, b = score.split(":")
    a, b = int(a), int(b)
    if fmt == "home:away":
        return a, b
    elif fmt == "away:home":
        return b, a
    else:
        raise ValueError("Unknown score format")

def batting_team_for_half(half: str, home_team: str, away_team: str) -> str:
    return away_team if half == "top" else home_team

def score_diff_for_batting(score: str, half: str, home_team: str, away_team: str, score_fmt: str = "home:away") -> int:
    """
    타자(공격) 팀 기준 점수차 = (타자 팀 득점 - 상대 득점)
    """
    home, away = parse_score_pair(score, fmt=score_fmt)
    batting_team = batting_team_for_half(half, home_team, away_team)
    if batting_team == home_team:
        return home - away
    else:
        return away - home

def text_has_any(s: Optional[str], keywords: tuple) -> bool:
    if not s:
        return False
    return any(kw in s for kw in keywords)

def detect_triggers(atbat: Dict[str, Any]) -> Dict[str, bool]:
    pinch = False
    ibb = False
    error = False

    # 1) 대타/대주자
    if atbat.get("original_batter") and atbat.get("actual_batter"):
        if atbat["original_batter"] != atbat["actual_batter"]:
            pinch = True
    # 텍스트 키워드
    if text_has_any(atbat.get("full_result"), KW_PINCH) or text_has_any(atbat.get("main_result"), KW_PINCH):
        pinch = True
    # pitch_sequence 중 event 텍스트도 훑기
    for ps in atbat.get("pitch_sequence", []):
        ev = ps.get("event")
        if isinstance(ev, str):
            if text_has_any(ev, KW_PINCH):
                pinch = True
        elif isinstance(ev, list):
            if any(text_has_any(x, KW_PINCH) for x in ev if isinstance(x, str)):
                pinch = True

    # 2) 고의4구
    if text_has_any(atbat.get("full_result"), KW_IBB) or text_has_any(atbat.get("main_result"), KW_IBB):
        ibb = True

    # 3) 실책
    if text_has_any(atbat.get("full_result"), KW_ERROR) or text_has_any(atbat.get("main_result"), KW_ERROR):
        error = True

    return {
        "pinch_event": pinch,
        "intentional_walk": ibb,
        "error_happened": error,
    }

def atbat_to_pli_payload(
    atbat: Dict[str, Any],
    context: Dict[str, Any],
    *,
    temp_c: Optional[float] = None,
    base_wpa: Optional[float] = None,
    loss_streak: Optional[int] = None,
    score_format: str = "home:away",
) -> Dict[str, Any]:
    inning = int(atbat["inning"])
    condition = calculate_condition(atbat.get("actual_batter"))
    outs = int(atbat["out"])
    half = atbat.get("half", "top")
    runners_code = runners_code_from_onbase(atbat.get("on_base", {}))
    score_str = atbat.get("score", "0:0")

    score_diff = score_diff_for_batting(
        score_str, half, context["home_team"], context["away_team"], score_fmt=score_format
    )
    triggers = detect_triggers(atbat)
    bwpa = float(base_wpa if base_wpa is not None else 0.12)

    return {
        # === 식별자(타자/투수/게임/타석) 유지 ===
        "game_id": context.get("game_id"),
        "batter_id": atbat.get("actual_batter"),
        "pitcher_id": atbat.get("pitcher"),
        "inning": inning,
        "half": half,
        "bat_order": atbat.get("bat_order"),
        # === 상황 입력 ===
        "outs": outs,
        "runners_code": runners_code,
        "score_diff": score_diff,
        "base_wpa": bwpa,
        # === 외부 요인(초기값 None이면 weight=1.0) ===
        "temp_c": temp_c,
        "loss_streak": loss_streak,
        "condition":condition,
        # === 트리거 ===
        "pinch_event": triggers["pinch_event"],
        "intentional_walk": triggers["intentional_walk"],
        "error_happened": triggers["error_happened"],
        "next_batter_after_error": False,
    }

def inning_pack_to_pli_requests(feed: Dict[str, Any], *, temp_c: Optional[float] = None, score_format: str = "home:away") -> List[Dict[str, Any]]:
    data = feed["data"]
    ctx = {
        "home_team": data["defense_positions"]["home_team"],
        "away_team": data["defense_positions"]["away_team"],
    }
    requests: List[Dict[str, Any]] = []

    def _absorb(side: str):
        side_block = data.get(side)
        if not side_block:
            return
        inning_default = side_block.get("inning_number")
        atbats = side_block.get("atbats", [])

        for idx, ab in enumerate(atbats, start=1):
            # half 기본값: side(top/bot)
            ab_half = ab.get("half") or side
            # bat_order 기본값: 원본 bat_order → 없으면 appearance_number → 없으면 enumerate 인덱스
            ab_bat_order = ab.get("bat_order")
            if ab_bat_order in (None, ""):
                ab_bat_order = ab.get("appearance_number") or idx

            # inning 기본값: 각 atbat의 inning → 없으면 side_block의 inning_number
            ab_inning = ab.get("inning") or inning_default

            # 식별자 주입
            ab = {
                **ab,
                "half": ab_half,
                "bat_order": ab_bat_order,
                "inning": ab_inning,
            }

            req = atbat_to_pli_payload(
                ab, ctx, temp_c=temp_c, score_format=score_format
            )
            requests.append(req)

    _absorb("top")
    _absorb("bot")
    return requests

def inning_pack_to_pli_scores(
    feed: dict,                        # ✅ 입력: 한 이닝의 raw 데이터 패킷 (top/bot at-bats 포함)
    *,
    temp_c: float | None = None,       # 날씨 요인 (섭씨 온도). 없으면 None.
    loss_streak: int | None = None,    # 팀의 연패 횟수 (없으면 None).
    recent_ba: float | None = None,    # 타자의 최근 타율 (없으면 None).
    season_ba: float | None = None,    # 타자의 시즌 타율 (없으면 None).
    score_format: str = "home:away",   # 점수 문자열 형식 ("home:away" 또는 "away:home").
    return_detail: bool = False,       # 반환 모드 (False: 단순 pli 값만, True: 상세 조건 포함)
):
    """
    return_detail=False: [{"inning":3,"half":"top","bat_order":2,"pli":0.1372}, ...]
    return_detail=True : [{"pli":..., "weights":..., "situation":..., "input":...}, ...]
    """
    reqs = inning_pack_to_pli_requests(feed, temp_c=temp_c, score_format=score_format)
    results = []
    for r in reqs:
        level, desc, total_score = calculate_situation_level(
            r["inning"], r["outs"], r["runners_code"], r["score_diff"]
        )
        w_env = batting_weather_weight(r.get("temp_c"))
        w_personal = combine_weights(
            condition_weight(r.get("recent_ba"), r.get("season_ba")),
        )
        w_situ = combine_weights(
            streak_weight(r.get("loss_streak")),
            pinch_weight(bool(r.get("pinch_event"))),
            ibb_focus_weight(bool(r.get("intentional_walk"))),
            error_momentum_weight(bool(r.get("error_happened")), bool(r.get("next_batter_after_error")))
        )
        w_total = combine_weights(w_env, w_personal, w_situ)
        pli_val = round(float(r["base_wpa"]) * w_total, 4)

        if return_detail:
            results.append({
                "input": r,
                "pli": pli_val,
                "weights": {
                    "environment": round(w_env, 4),
                    "personal": round(w_personal, 4),
                    "situational": round(w_situ, 4),
                    "combined": round(w_total, 4),
                },
                "situation": {"level": level, "description": desc, "score": total_score},
            })
        else:
            results.append({
                "inning": r["inning"],
                "half": r.get("half"),
                "bat_order": r.get("bat_order"),
                "pli": pli_val,
            })
    return results

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

In [51]:
scode = "CW"
outputs = []
for ab in raw_data["data"]["top"]["atbats"]:
    triggers = detect_triggers(ab)
    runners = runners_code_from_onbase(ab.get("on_base", {}))
    score_diff = score_diff_for_batting(
        ab.get("score", "0:0"), ab.get("half", "top"),
        raw_data["data"]["defense_positions"]["home_team"],
        raw_data["data"]["defense_positions"]["away_team"],
        score_fmt="home:away",
    )
    row = {
        "inning": int(ab["inning"]),
        "outs": int(ab["out"]),
        "runners_code": runners,
        "score_diff": score_diff,
        "base_wpa": 0.12,
        "temp_c": 24.0,
        "loss_streak": None,
        "recent_ba": None,
        "season_ba": None,
        "years_pro": None,
        "pinch_event": triggers["pinch_event"],
        "intentional_walk": triggers["intentional_walk"],
        "error_happened": triggers["error_happened"],
        "next_batter_after_error": False,
    }
    outputs.append(compute_pli_row(pd.Series(row)))
for o in outputs:
    print(o)

{'pli': 0.131, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.0, 'w_total': 1.092, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 6}
{'pli': 0.2359, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.8, 'w_total': 1.9656, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 7}
{'pli': 0.2359, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.8, 'w_total': 1.9656, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 7}
{'pli': 0.2359, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.8, 'w_total': 1.9656, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 6}
{'pli': 0.131, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.0, 'w_total': 1.092, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 6}
{'pli': 0.131, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.0, 'w_total': 1.092, 'level': 2, 'level_desc': '관리 필요', 'level_score': 3}
{'pli': 0.131, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.0, 'w_total': 1.092, 'level': 2, 'level_desc': '관리 필요', 'level_score': 4}
{'pli': 0.131, 'w_env': 1.04, 'w_person

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

## 더미데이터

In [18]:
import json

raw_data = json.loads("""
{
    "status": "OK_REALTIME",
    "message": "9회 이닝 정보 (실시간)",
    "data": {
        "top": {
            "game": "20250815HHNC02025",
            "inning_number": 9,
            "atbats": [
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "52992",
                    "bat_order": 8,
                    "original_batter": null,
                    "actual_batter": "78288",
                    "out": "0",
                    "score": "6:2",
                    "on_base": {
                        "base1": "8",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.241,
                        1.572,
                        0.75,
                        -0.75
                    ],
                    "full_result": "9회초 8번타순 10구 체크스윙 → 노 스윙 : 한화 최재훈",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.24329661094573862,
                                    2.5761973494323676
                                ]
                            ],
                            "speed": "143",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.8336798135282728,
                                    1.5261444742144614
                                ]
                            ],
                            "speed": "145",
                            "count": "0-2",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    1.4529255546021904,
                                    2.559007338995863
                                ]
                            ],
                            "speed": "143",
                            "count": "1-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.702147611301194,
                                    1.8233275413919001
                                ]
                            ],
                            "speed": "131",
                            "count": "1-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    1.4210670945795632,
                                    3.7663946955403604
                                ]
                            ],
                            "speed": "145",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 6,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.8095409322745795,
                                    3.9387431703955658
                                ]
                            ],
                            "speed": "129",
                            "count": "3-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 7,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.02783243960664472,
                                    1.8781848788812106
                                ]
                            ],
                            "speed": "133",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 8,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.05236525554944599,
                                    1.964196311108453
                                ]
                            ],
                            "speed": "134",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 9,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.3406670849886363,
                                    2.72577803018826
                                ]
                            ],
                            "speed": "133",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 10,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.11430030395912949,
                                    3.670121478893454
                                ]
                            ],
                            "speed": "145",
                            "count": "4-2",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ],
                    "main_result": "볼넷"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "52992",
                    "bat_order": 9,
                    "original_batter": null,
                    "actual_batter": "64006",
                    "out": "0",
                    "score": "6:2",
                    "on_base": {
                        "base1": "9",
                        "base2": "8",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.34,
                        1.62,
                        0.75,
                        -0.75
                    ],
                    "full_result": "1루주자 하주석 : 2루까지 진루",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.08564469843110101,
                                    2.9387716888482194
                                ]
                            ],
                            "speed": "145",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": "1루주자 최재훈 : 대주자 하주석 (으)로 교체|투수 투수판 이탈"
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -1.8054607871604451,
                                    3.327352459336841
                                ]
                            ],
                            "speed": "131",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ],
                    "main_result": "몸에 맞는 볼"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "52992",
                    "bat_order": 1,
                    "original_batter": null,
                    "actual_batter": "52764",
                    "out": "0",
                    "score": "6:2",
                    "on_base": {
                        "base1": "9",
                        "base2": "8",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": null,
                    "full_result": "1번타자 손아섭 : 대타 허인서 (으)로 교체",
                    "pitch_sequence": []
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "52992",
                    "bat_order": 1,
                    "original_batter": null,
                    "actual_batter": "52764",
                    "out": "2",
                    "score": "6:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "0",
                        "base3": "8"
                    },
                    "appearance_number": 2,
                    "strike_zone": [
                        3.365,
                        1.632,
                        0.75,
                        -0.75
                    ],
                    "full_result": "대타 허인서|허인서 : 2루수 병살타 아웃 (2루수->유격수->1루수 송구아웃)|1루주자 심우준 : 포스아웃 (2루수->유격수 2루 터치아웃)|2루주자 하주석 : 3루까지 진루",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.047695971814526644,
                                    1.4932743878739583
                                ]
                            ],
                            "speed": "132",
                            "count": "0-1",
                            "pitch_result": "헛스윙",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.6223723129303991,
                                    1.6517441885892956
                                ]
                            ],
                            "speed": "131",
                            "count": "0-2",
                            "pitch_result": "헛스윙",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.2921863157416469,
                                    0.6272210678100341
                                ]
                            ],
                            "speed": "132",
                            "count": "0-2",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 2,
                    "original_batter": null,
                    "actual_batter": "55703",
                    "out": "2",
                    "score": "6:2",
                    "on_base": {
                        "base1": "2",
                        "base2": "0",
                        "base3": "8"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.417,
                        1.657,
                        0.75,
                        -0.75
                    ],
                    "full_result": "리베라토 : 볼넷",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.11781421914619217,
                                    1.3863201510485417
                                ]
                            ],
                            "speed": "136",
                            "count": "1-0",
                            "pitch_result": "볼",
                            "event": "투수 이준혁 : 투수 최성영 (으)로 교체"
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.964361435484117,
                                    3.2092061357393193
                                ]
                            ],
                            "speed": "123",
                            "count": "2-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.24882700755065246,
                                    1.9338915108629728
                                ]
                            ],
                            "speed": "139",
                            "count": "2-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -1.608706566191269,
                                    0.5425127843235655
                                ]
                            ],
                            "speed": "125",
                            "count": "3-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.8152578474425323,
                                    1.1853016646577343
                                ]
                            ],
                            "speed": "139",
                            "count": "4-1",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ],
                    "main_result": "볼넷"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 3,
                    "original_batter": null,
                    "actual_batter": "53764",
                    "out": "2",
                    "score": "9:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.17,
                        1.537,
                        0.75,
                        -0.75
                    ],
                    "full_result": "9회초 3번타순 2구 체크스윙 → 노 스윙 : 한화 문현빈|문현빈 : 우익수 뒤 홈런 (홈런거리:120M)|1루주자 리베라토 : 홈인|3루주자 하주석 : 홈인",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.13216487701923674,
                                    2.7162858254605293
                                ]
                            ],
                            "speed": "141",
                            "count": "0-1",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.27346128928310254,
                                    3.467988919921652
                                ]
                            ],
                            "speed": "141",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.339184922597328,
                                    2.288975429188571
                                ]
                            ],
                            "speed": "138",
                            "count": "1-1",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 4,
                    "original_batter": null,
                    "actual_batter": "69737",
                    "out": "2",
                    "score": "9:2",
                    "on_base": {
                        "base1": "4",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.349,
                        1.624,
                        0.75,
                        -0.75
                    ],
                    "full_result": "노시환 : 좌익수 오른쪽 1루타",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.4912340118019476,
                                    1.7541685091493786
                                ]
                            ],
                            "speed": "141",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    0.8009185282829967,
                                    2.2209603793411383
                                ]
                            ],
                            "speed": "124",
                            "count": "0-1",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "좌익수 오른쪽 1루타"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 5,
                    "original_batter": null,
                    "actual_batter": "66704",
                    "out": "2",
                    "score": "9:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "5",
                        "base3": "4"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.259,
                        1.581,
                        0.75,
                        -0.75
                    ],
                    "full_result": "1루주자 노시환 : 3루까지 진루",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.43800618849095563,
                                    3.3611676205341436
                                ]
                            ],
                            "speed": "140",
                            "count": "1-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.07282903328279144,
                                    2.086065389188862
                                ]
                            ],
                            "speed": "138",
                            "count": "1-0",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "좌익수 왼쪽 2루타"
                },
                {
                    "inning": 9,
                    "half": "top",
                    "pitcher": "66920",
                    "bat_order": 6,
                    "original_batter": null,
                    "actual_batter": "68700",
                    "out": "3",
                    "score": "9:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "5",
                        "base3": "4"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.254,
                        1.578,
                        0.75,
                        -0.75
                    ],
                    "full_result": "이원석 : 삼진 아웃",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.9125483887184911,
                                    5.052954664524453
                                ]
                            ],
                            "speed": "121",
                            "count": "1-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.4033114303068699,
                                    3.532009513756412
                                ]
                            ],
                            "speed": "141",
                            "count": "2-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.5328251215007548,
                                    1.3475485544128083
                                ]
                            ],
                            "speed": "140",
                            "count": "3-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.07957533617980395,
                                    2.879098109221778
                                ]
                            ],
                            "speed": "136",
                            "count": "3-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.03264936231069082,
                                    1.9881459752132558
                                ]
                            ],
                            "speed": "139",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 6,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    0.4200739295574205,
                                    1.8566700453094453
                                ]
                            ],
                            "speed": "122",
                            "count": "3-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 7,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.7128730572276658,
                                    1.4477182132911104
                                ]
                            ],
                            "speed": "140",
                            "count": "3-3",
                            "pitch_result": "스트라이크",
                            "event": null
                        }
                    ],
                    "main_result": "삼진 아웃"
                }
            ]
        },
        "bot": {
            "game": "20250815HHNC02025",
            "inning_number": 9,
            "atbats": [
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 6,
                    "original_batter": null,
                    "actual_batter": "69992",
                    "out": "0",
                    "score": "9:2",
                    "on_base": {
                        "base1": "0",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": null,
                    "full_result": "대타 허인서 : 포수(으)로 수비위치 변경|6번타자 이우성 : 대타 최정원 (으)로 교체",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "event": [
                                "대주자 하주석 : 투수 한승혁 (으)로 교체"
                            ]
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 6,
                    "original_batter": null,
                    "actual_batter": "69992",
                    "out": "0",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 2,
                    "strike_zone": [
                        3.245,
                        1.574,
                        0.75,
                        -0.75
                    ],
                    "full_result": "대타 최정원|최정원 : 몸에 맞는 볼",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.27497515960021657,
                                    3.0290136243513848
                                ]
                            ],
                            "speed": "133",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -0.5500841406573953,
                                    2.637500512695186
                                ]
                            ],
                            "speed": "133",
                            "count": "0-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -1.682706606376398,
                                    4.230288681537843
                                ]
                            ],
                            "speed": "135",
                            "count": "1-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.633582700382768,
                                    2.5280461324174577
                                ]
                            ],
                            "speed": "146",
                            "count": "1-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    1.0796329384896899,
                                    -0.8945528020958866
                                ]
                            ],
                            "speed": "133",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 7,
                    "original_batter": null,
                    "actual_batter": "69995",
                    "out": "0",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": null,
                    "full_result": "7번타자 김휘집 : 대타 서호철 (으)로 교체",
                    "pitch_sequence": []
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 7,
                    "original_batter": null,
                    "actual_batter": "69995",
                    "out": "1",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 2,
                    "strike_zone": [
                        3.329,
                        1.615,
                        0.75,
                        -0.75
                    ],
                    "full_result": "대타 서호철|서호철 : 중견수 플라이 아웃",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.7218203864624638,
                                    2.284833535689379
                                ]
                            ],
                            "speed": "133",
                            "count": "0-1",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.7313573529684394,
                                    2.1760150251423602
                                ]
                            ],
                            "speed": "133",
                            "count": "0-2",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    1.4284755369128663,
                                    3.4859238390085614
                                ]
                            ],
                            "speed": "146",
                            "count": "1-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "커브",
                            "pitch_coordinate": [
                                [
                                    1.7014443895057707,
                                    1.2493566991948706
                                ]
                            ],
                            "speed": "115",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.17698295647015272,
                                    2.6440611088373487
                                ]
                            ],
                            "speed": "132",
                            "count": "2-2",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ]
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 8,
                    "original_batter": null,
                    "actual_batter": "50901",
                    "out": "2",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.323,
                        1.612,
                        0.75,
                        -0.75
                    ],
                    "full_result": "안인산 : 중견수 플라이 아웃",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.3296011994438843,
                                    2.4564535892088823
                                ]
                            ],
                            "speed": "145",
                            "count": "0-0",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "중견수 플라이 아웃"
                },
                {
                    "inning": 9,
                    "half": "bot",
                    "pitcher": "61666",
                    "bat_order": 9,
                    "original_batter": null,
                    "actual_batter": "64022",
                    "out": "3",
                    "score": "9:2",
                    "on_base": {
                        "base1": "6",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.237,
                        1.57,
                        0.75,
                        -0.75
                    ],
                    "full_result": "안중열 : 삼진 아웃",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.3214032311851819,
                                    1.7998801806888647
                                ]
                            ],
                            "speed": "133",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "커브",
                            "pitch_coordinate": [
                                [
                                    2.6253715818215024,
                                    -0.6299354659471188
                                ]
                            ],
                            "speed": "114",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    1.1604821582069043,
                                    1.8609382262776664
                                ]
                            ],
                            "speed": "134",
                            "count": "2-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.8236813571019375,
                                    1.5596565704220733
                                ]
                            ],
                            "speed": "145",
                            "count": "2-2",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.17136184027644394,
                                    1.550312804451011
                                ]
                            ],
                            "speed": "134",
                            "count": "2-3",
                            "pitch_result": "스트라이크",
                            "event": null
                        }
                    ],
                    "main_result": "삼진 아웃"
                }
            ]
        },
        "defense_positions": {
            "home_team": "NC",
            "away_team": "HH",
            "home": {
                "유격수": "김주원",
                "중견수": "",
                "2루수": "박민우",
                "1루수": "데이비슨",
                "우익수": "박건우",
                "지명타자": "안인산",
                "좌익수": "",
                "3루수": "김휘집",
                "포수": "안중열",
                "투수": "최성영"
            },
            "away": {
                "포수": "최재훈",
                "지명타자": "손아섭",
                "중견수": "리베라토",
                "좌익수": "문현빈",
                "3루수": "노시환",
                "1루수": "채은성",
                "우익수": "이진영",
                "2루수": "안치홍",
                "투수": "김범수",
                "11": "하주석",
                "유격수": "심우준"
            }
        }
    }
}""")

## test

In [ ]:
compute_pli_row()

In [50]:
results = inning_pack_to_pli_scores(raw_data)

for r in results:
    print(r)

{'inning': 9, 'half': 'top', 'bat_order': 8, 'pli': 0.12}
{'inning': 9, 'half': 'top', 'bat_order': 9, 'pli': 0.216}
{'inning': 9, 'half': 'top', 'bat_order': 1, 'pli': 0.216}
{'inning': 9, 'half': 'top', 'bat_order': 1, 'pli': 0.216}
{'inning': 9, 'half': 'top', 'bat_order': 2, 'pli': 0.12}
{'inning': 9, 'half': 'top', 'bat_order': 3, 'pli': 0.12}
{'inning': 9, 'half': 'top', 'bat_order': 4, 'pli': 0.12}
{'inning': 9, 'half': 'top', 'bat_order': 5, 'pli': 0.12}
{'inning': 9, 'half': 'top', 'bat_order': 6, 'pli': 0.12}
{'inning': 9, 'half': 'bot', 'bat_order': 6, 'pli': 0.216}
{'inning': 9, 'half': 'bot', 'bat_order': 6, 'pli': 0.216}
{'inning': 9, 'half': 'bot', 'bat_order': 7, 'pli': 0.216}
{'inning': 9, 'half': 'bot', 'bat_order': 7, 'pli': 0.216}
{'inning': 9, 'half': 'bot', 'bat_order': 8, 'pli': 0.12}
{'inning': 9, 'half': 'bot', 'bat_order': 9, 'pli': 0.12}


In [52]:
# years_pro 항목 제거 버전으로 재정의

def compute_pli_row(row: pd.Series) -> Dict[str, Any]:
    # 상황 레벨
    level, desc, score = calculate_situation_level(
        int(row['inning']), int(row['outs']), str(row['runners_code']), int(row['score_diff'])
    )
    # 가중치
    w_env = batting_weather_weight(row.get('temp_c'))
    w_personal = combine_weights(
        condition_weight(row.get('recent_ba'), row.get('season_ba')),
    )
    w_situ = combine_weights(
        streak_weight(row.get('loss_streak')),
        pinch_weight(bool(row.get('pinch_event', False))),
        ibb_focus_weight(bool(row.get('intentional_walk', False))),
        error_momentum_weight(bool(row.get('error_happened', False)), bool(row.get('next_batter_after_error', False)))
    )
    pf = STADIUM_PARK.get(scode)
    w_park = parkfactor_weight(pf)
    w_total = combine_weights(w_env, w_personal, w_situ, w_park)
    pli = float(row['base_wpa']) * w_total
    return {
        "pli": round(pli, 4),
        "w_env": round(w_env, 4),
        "w_personal": round(w_personal, 4),
        "w_situ": round(w_situ, 4),
        "w_total": round(w_total, 4),
        "level": level, "level_desc": desc, "level_score": score
    }

def inning_pack_to_pli_scores(
    feed: dict,
    *,
    temp_c: float | None = None,
    loss_streak: int | None = None,
    recent_ba: float | None = None,
    season_ba: float | None = None,
    score_format: str = "home:away",
    return_detail: bool = False,
):
    reqs = inning_pack_to_pli_requests(feed, temp_c=temp_c, score_format=score_format)
    results = []
    for r in reqs:
        level, desc, total_score = calculate_situation_level(
            r["inning"], r["outs"], r["runners_code"], r["score_diff"]
        )
        w_env = batting_weather_weight(r.get("temp_c"))
        w_personal = combine_weights(
            condition_weight(r.get("recent_ba"), r.get("season_ba")),
        )
        w_situ = combine_weights(
            streak_weight(r.get("loss_streak")),
            pinch_weight(bool(r.get("pinch_event"))),
            ibb_focus_weight(bool(r.get("intentional_walk"))),
            error_momentum_weight(bool(r.get("error_happened")), bool(r.get("next_batter_after_error")))
        )
        pf = STADIUM_PARK.get(scode)
        w_park = parkfactor_weight(pf)
        w_total = combine_weights(w_env, w_personal, w_situ, w_park)
        pli_val = round(float(r["base_wpa"]) * w_total, 4)

        if return_detail:
            results.append({
                "input": r,
                "pli": pli_val,
                "weights": {
                    "environment": round(w_env, 4),
                    "personal": round(w_personal, 4),
                    "situational": round(w_situ, 4),
                    "combined": round(w_total, 4),
                },
                "situation": {"level": level, "description": desc, "score": total_score},
            })
        else:
            results.append({
                "inning": r["inning"],
                "half": r.get("half"),
                "bat_order": r.get("bat_order"),
                "pli": pli_val,
            })
    return results

In [53]:
scode = "CW"
outputs = []
for ab in raw_data["data"]["top"]["atbats"]:
    triggers = detect_triggers(ab)
    runners = runners_code_from_onbase(ab.get("on_base", {}))
    score_diff = score_diff_for_batting(
        ab.get("score", "0:0"), ab.get("half", "top"),
        raw_data["data"]["defense_positions"]["home_team"],
        raw_data["data"]["defense_positions"]["away_team"],
        score_fmt="home:away",
    )
    row = {
        "inning": int(ab["inning"]),
        "outs": int(ab["out"]),
        "runners_code": runners,
        "score_diff": score_diff,
        "base_wpa": 0.12,
        "temp_c": 24.0,
        "loss_streak": None,
        "recent_ba": None,
        "season_ba": None,
        "pinch_event": triggers["pinch_event"],
        "intentional_walk": triggers["intentional_walk"],
        "error_happened": triggers["error_happened"],
        "next_batter_after_error": False,
    }
    outputs.append(compute_pli_row(pd.Series(row)))
for o in outputs:
    print(o)

{'pli': 0.131, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.0, 'w_total': 1.092, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 6}
{'pli': 0.2359, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.8, 'w_total': 1.9656, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 7}
{'pli': 0.2359, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.8, 'w_total': 1.9656, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 7}
{'pli': 0.2359, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.8, 'w_total': 1.9656, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 6}
{'pli': 0.131, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.0, 'w_total': 1.092, 'level': 3, 'level_desc': '중요한 승부처', 'level_score': 6}
{'pli': 0.131, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.0, 'w_total': 1.092, 'level': 2, 'level_desc': '관리 필요', 'level_score': 3}
{'pli': 0.131, 'w_env': 1.04, 'w_personal': 1.0, 'w_situ': 1.0, 'w_total': 1.092, 'level': 2, 'level_desc': '관리 필요', 'level_score': 4}
{'pli': 0.131, 'w_env': 1.04, 'w_person

In [ ]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha